# Wikipedia data Collection using SPARQLWrapper :

## The goal of this project is the investigate "How event driven is Wikipedia?"
- To reach a conclusion I can look for points of anomalies in the time series plot and look for a justification from another event dataframe (main events, birthdays , celebrations , deaths , release dates ...)


- Problem for this project is that in order to answer this question we need to evaluate wikipedia as a whole website not just select some articles
- Solution:  An idea could be to generate 10000 of  datatframes (people , events , science , art , dates ...) and then evaluate the time series dataframes.
- We can also have a language feature (fr, en , de ...) and then evaluate the user behaviour when it comes to language.

## The goal of this Notebook is Data collection:


First phase of the project is the data collection phase. For this we must have define different categories for our searches for example: list of people, events, dates, object…

- Specifically we want to select classes  in each of the defined cathegories: disasters, culture.
    - disasters will contain 3 classes: air accidents, Train collisions, helicopters
        - air accidents will contain the names of the air accidents that occured from 2015 to this day
    - culture will contain 3 classes: films, actors, charactors
        - each class will contain apropriate events that occured from 2015 to this day
        
In order to generate these terms I  used https://query.wikidata.org/  to extract the reseach query 

Get items to search: https://query.wikidata.org/querybuilder/?uselang=en


The next step is to extract the time series view data for each turm I used Analytics/AQS/Pageviews form the pything lib pageviewapi: 
https://pypi.org/project/pageviewapi/


- This page documents the Pageview API (v1), you can get pageview trends on specific articles or projects; filter by agent type or access method, and choose different time ranges and granularities; you can also get the most viewed articles of a certain project and timespan, and even check out the countries that visit a project the most. 

- Have different languages (en, fr, ...)
 

## Libs

In [2]:
#import wikipediaapi
import pageviewapi
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import requests
import sys
from SPARQLWrapper import SPARQLWrapper, JSON

# import the datetime module
import datetime
from datetime import datetime
pd.options.mode.chained_assignment = None  # default='warn'

# Testing the  pageviewapi


In [3]:
pageview_count=pageviewapi.aggregate('en.wikipedia', '2020100100', '2020103100', access='all-access',
                      agent='all-agents', granularity='daily')

In [4]:
most_viewed=pageviewapi.top('en.wikipedia', 2020, 11, 14, access='all-access')

In [5]:
most_viewed2020=[i["article"] for i in most_viewed["items"][0]["articles"]]

# Data collection

In [124]:
endpoint_url = "https://query.wikidata.org/sparql"

def get_results(endpoint_url, query):
    user_agent = "WDQS-example Python/%s.%s" % (sys.version_info[0], sys.version_info[1])
    # TODO adjust user agent; see https://w.wiki/CX6
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()

In [125]:
class_dic={
    #"Fictional_characters":[],
    #"Actors":[],
    #"Movies":[],
   # "Countries":[],
    #"Capitals":[],
    #"Billionaires":[],
    #"Universities":[],
    "air_accidents":[],
    "train_accident" :[],
    "Volcano_eruptions":[],
    "Cyclones":[],
    "Earthquakes":[],
}

In [126]:
query_dic={
     "Earthquakes": [
            """ #Earthquakes
                SELECT ?label ?date  #pointin time
        WHERE
        {
           ?subj wdt:P31 wd:Q7944  . 
           ?subj rdfs:label ?label ;
                 wdt:P585  ?date. 
                 FILTER (lang(?label) = "en") }"""
            ],
    
    
    "Cyclones": [
            """ #Hurricanes 
                SELECT  ?date ?subj ?label
        WHERE
        {
           ?subj wdt:P31 wd:Q79602  . 
           ?subj rdfs:label ?label ;
                 wdt:P585  ?date.            
           FILTER (lang(?label) = "en") }"""
            ],
    
    
    "Volcano_eruptions": [
            """#Volcano_eruptions 
                SELECT  ?label ?date
        WHERE
        {
           ?subj wdt:P31 wd:Q7692360  . 
           ?subj rdfs:label ?label ;
                 wdt:P585  ?date. 
                 
          FILTER (lang(?label) = "en") }"""
            ],
    
    
    "train_accident" : ["""#train accident 
        SELECT  ?label ?date
        WHERE
        {
           ?subj wdt:P31 wd:Q1078765  .
           ?subj rdfs:label ?label;
                 wdt:P585  ?date. 
        FILTER (lang(?label) = "en")
        
        }""" ],
    
    
    "air_accidents" : [""" #Locations of air accidents
        SELECT ?label ?coord ?place ?date
        WHERE
        {
           ?subj wdt:P31 wd:Q744913  .
           ?subj wdt:P625 ?coord .
           ?subj rdfs:label ?label ;
                 wdt:P585  ?date. 
         FILTER (lang(?label) = "en")
        }"""],
    
    
#     "Fictional_characters":
    
#     ["""SELECT ?char ?charName (GROUP_CONCAT(DISTINCT ?typeLabel;separator=", ") AS ?types) (GROUP_CONCAT(DISTINCT ?universeLabel;separator=", ") AS ?universes)
#         WHERE {
#           ?char wdt:P1080 wd:Q931597;
#                   wdt:P31 ?type ;
#                   wdt:P1080 ?universe .
#           SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en".
#                                  ?char rdfs:label ?charName .
#                                  ?universe rdfs:label ?universeLabel .
#                                  ?type rdfs:label ?typeLabel .}
#         } GROUP BY ?char ?charName
#         LIMIT 500"""],
    
    
#     "Actors":
    
#     ["""
#         SELECT ?human ?humanLabel WHERE
#         {
#           ?human wdt:P31 wd:Q5
#           ; wdt:P106 wd:Q33999 .
#           ?human wdt:P18 ?picture .
#           OPTIONAL { ?human wdt:P569 ?dob . ?human wdt:P570 ?dod }.
#           BIND(YEAR(?dob) as ?yob) . #if available: year
#           BIND(YEAR(?dod) as ?yod) .
#           SERVICE wikibase:label {bd:serviceParam wikibase:language "en" .  }
#         }
#         LIMIT 500"""],
    
#     "Movies":
#             ["""#Movies released SINCE 2015
#         SELECT DISTINCT ?item ?itemLabel  WHERE {
#           ?item wdt:P31 wd:Q11424.
#           ?item wdt:P577 ?pubdate.
#           FILTER((?pubdate >= "2015-01-01T00:00:00Z"^^xsd:dateTime))
#           SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
#         }LIMIT 500"""
#         ],
    
    
#     "Countries":
    
#         ["""#List of present-day countries and capital(s)
#         SELECT DISTINCT ?country ?countryLabel ?capital ?capitalLabel
#         WHERE
#         {
#           ?country wdt:P31 wd:Q3624078 .
#           #not a former country
#           FILTER NOT EXISTS {?country wdt:P31 wd:Q3024240}
#           #and no an ancient civilisation (needed to exclude ancient Egypt)
#           FILTER NOT EXISTS {?country wdt:P31 wd:Q28171280}
#           OPTIONAL { ?country wdt:P36 ?capital } .

#           SERVICE wikibase:label { bd:serviceParam wikibase:language "en" }
#             }
#             ORDER BY ?countryLabel"""],
    
        
#     "Capitals":
    
#         ["""#List of present-day countries and capital(s)
#         SELECT DISTINCT ?capital ?capitalLabel ?country ?countryLabel 
#         WHERE
#         {
#           ?country wdt:P31 wd:Q3624078 .
#           #not a former country
#           FILTER NOT EXISTS {?country wdt:P31 wd:Q3024240}
#           #and no an ancient civilisation (needed to exclude ancient Egypt)
#           FILTER NOT EXISTS {?country wdt:P31 wd:Q28171280}
#           OPTIONAL { ?country wdt:P36 ?capital } .

#           SERVICE wikibase:label { bd:serviceParam wikibase:language "en" }
#             }
#             ORDER BY ?countryLabel"""],
    
    
#     "Billionaires":
    
#         [ """
#         SELECT ?item ?itemLabel (MAX(?billion) as ?billions)
#         WHERE
#         {
#           ?item wdt:P2218 ?worth.
#           ?item wdt:P19 ?location .

#           FILTER(?worth>1000000000).
#           BIND(?worth/1000000000 AS ?billion).
#           SERVICE wikibase:label { bd:serviceParam wikibase:language "en,de". }
#         }
#         GROUP BY ?locationLabel ?item ?itemLabel
#         ORDER BY DESC(?billions)
#         LIMIT 500""" ],
    
    
#     "Universities":
    
#         ["""SELECT  ?educatedatLabel ?langLabel ?human ?humanLabel 
#         {
#           ?lang wdt:P31/wdt:P279* wd:Q9143 .
#           ?human wdt:P31 wd:Q5 .
#           { ?lang wdt:P287 ?human } UNION { ?lang wdt:P170 ?human } UNION { ?lang wdt:P943 ?human } UNION { ?lang wdt:P178 ?human } .

#           ?human wdt:P69 ?educatedat .
#           ?educatedat wdt:P625 ?coords .
#           SERVICE wikibase:label { bd:serviceParam wikibase:language "en,fr" }
#         }
#         LIMIT 500"""]
    
}

In [127]:
result[list(result.keys())[1]]["value"] 

'Point(-70.817 -19.642)'

In [128]:
for category in  class_dic:
    print(category)
    #print(query_dic[category][0])
    results = get_results(endpoint_url, query_dic[category][0])
    for result in results["results"]["bindings"]:
        if result['label']["value"] not in class_dic[category]: # remove duplicates
                #class_dic[category]['value'].append(result[list(result.keys())[2]]["value"])   
                #class_dic[category]['date'].append(result[list(result.keys())[0]]["value"])  
                class_dic[category].append([result['label']["value"],result['date']["value"]])   

    print(result['label']["value"],' of date ',result['date']["value"])
        
        

air_accidents
Tibet Airlines Flight 9833  of date  2022-05-12T00:00:00Z
train_accident
2022 Missouri train derailment  of date  2022-06-27T00:00:00Z
Volcano_eruptions
Campanian Ignimbrite eruption  of date  -40000-01-01T00:00:00Z
Cyclones
Cyclone Gaja  of date  2018-01-01T00:00:00Z
Earthquakes
2020 Elazığ earthquake  of date  2020-01-24T00:00:00Z


In [134]:
length_dict = {key: len(value) for key, value in class_dic.items()} #no limit or no liit then random select
length_dict

{'air_accidents': 1244,
 'train_accident': 535,
 'Volcano_eruptions': 41,
 'Cyclones': 64,
 'Earthquakes': 1331}

In [130]:
# initilizing my dataframe with a test example:

df_marks=pd.DataFrame()
#df_marks['Language']='lang'
df_marks['Category']='class'
df_marks['Event_first_revision_date']='date'
df_marks['Birthday']='Birthday'

url = 'https://en.wikipedia.org/w/api.php'


for key_cat in class_dic:
    print(key_cat,' ----------------------------------------')
    # pageviewapi per class
    for event in class_dic[key_cat]:    
        
        lang="en.wikipedia"  
        try:
            term=event[0]

            search_term=pageviewapi.per_article("en.wikipedia", term, '20150101', '20230101',
                            access='all-access', agent='user', granularity='daily')

            search_term_dic = [{'timestamp': x['timestamp'], 'views': x['views']} for x in search_term['items']]
            search_term_df=pd.DataFrame(search_term_dic).explode('views')
            search_term_df['timestamp'] = [datetime.strptime(x, "%Y%m%d%H")for x in search_term_df['timestamp']]
            search_term_df = search_term_df.set_index('timestamp')
            search_term_df.columns = [term]

            df_marks = df_marks.append(search_term_df.T, ignore_index=False)
            #df_marks.tail(1)['Language']="en.wikipedia"
            df_marks.tail(1)['Category']=key_cat

            params = {
                    'action': 'query',
                    'format': 'json',
                    'rvprop' : 'timestamp',
                    'titles': term,
                    'prop': 'revisions',
                    'rvdir' : 'newer',
                    'rvlimit' : '1'            }

            data = requests.get(url, params=params).json()
            key=[i for i in data['query']['pages'].keys()]
            df_marks.tail(1)['Event_first_revision_date']=data['query']['pages'][key[0]]['revisions'][0]['timestamp']
            df_marks.tail(1)['Birthday']=event[1]



            df_marks=df_marks.fillna(0)
        except:
            #print('Excep:', term ,' - ', end ="")
            pass
    print('  ')


air_accidents  ----------------------------------------
  
train_accident  ----------------------------------------
  
Volcano_eruptions  ----------------------------------------
  
Cyclones  ----------------------------------------
  
Earthquakes  ----------------------------------------
  


In [131]:
df_marks

,2015-07-01 00:00:00,2015-07-02 00:00:00,2015-07-03 00:00:00,2015-07-04 00:00:00,2015-07-05 00:00:00,2015-07-06 00:00:00,2015-07-07 00:00:00,2015-07-08 00:00:00,2015-07-09 00:00:00,2015-07-10 00:00:00,...,2022-10-01 00:00:00,2022-10-02 00:00:00,2022-10-03 00:00:00,2022-10-04 00:00:00,2022-10-05 00:00:00,2022-10-06 00:00:00,2022-10-07 00:00:00,Birthday,Category,Event_first_revision_date
China Airlines Flight 006,216.0,199.0,253.0,160.0,222.0,189.0,172.0,210.0,372.0,355.0,...,276.0,189.0,157.0,176.0,297.0,742.0,231.0,1985-02-19T00:00:00Z,air_accidents,2005-07-31T21:39:47Z
China Airlines Flight 676,82.0,99.0,58.0,79.0,86.0,93.0,80.0,75.0,65.0,62.0,...,64.0,62.0,66.0,45.0,65.0,65.0,57.0,1998-02-16T00:00:00Z,air_accidents,2005-01-18T13:19:25Z
South African Airways Flight 295,243.0,222.0,189.0,216.0,214.0,328.0,232.0,251.0,319.0,237.0,...,164.0,239.0,222.0,223.0,241.0,191.0,428.0,1987-11-28T00:00:00Z,air_accidents,2004-07-03T05:47:43Z
2008 Los Roques archipelago Transaven Let L-410 crash,7.0,2.0,3.0,13.0,5.0,7.0,9.0,5.0,8.0,2.0,...,3.0,5.0,8.0,3.0,3.0,1.0,7.0,2008-01-04T00:00:00Z,air_accidents,2008-01-06T09:11:59Z
1972 Andes flight disaster,1155.0,2258.0,3151.0,2528.0,2490.0,2577.0,2999.0,3461.0,3680.0,4011.0,...,20.0,12.0,17.0,22.0,15.0,12.0,21.0,1972-10-13T00:00:00Z,air_accidents,2016-12-01T03:35:09Z
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019 Albania earthquake,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,61.0,44.0,40.0,59.0,59.0,56.0,46.0,2019-11-26T00:00:00Z,Earthquakes,2019-11-26T15:08:55Z
2019 Davao del Sur earthquake,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,32.0,20.0,20.0,27.0,30.0,16.0,27.0,2019-12-15T00:00:00Z,Earthquakes,2019-12-15T16:29:41Z
2020 Puerto Rico earthquake,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2020-01-07T00:00:00Z,Earthquakes,2020-01-11T18:04:34Z
2020 Kashgar earthquake,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,5.0,0.0,1.0,2.0,4.0,2020-01-19T00:00:00Z,Earthquakes,2021-05-29T08:00:01Z


In [133]:
df_marks['Category'].value_counts()

air_accidents        1163
Earthquakes          1087
train_accident        384
Cyclones               56
Volcano_eruptions      20
Name: Category, dtype: int64

In [122]:
df_marks.to_csv('data_1.csv',index=True) 

## test

In [7]:
query = """#Locations of air accidents
SELECT ?label ?coord ?place ?date
WHERE
{
   ?subj wdt:P31 wd:Q744913  .
   ?subj wdt:P625 ?coord .
   ?subj rdfs:label ?label;
    wdt:P585  ?date. 
                 FILTER (lang(?label) = "en")
}"""

In [59]:
result['label']['value']

'1947 Cape Sable hurricane'

In [62]:

query_results_Moovies = """

SELECT ?char ?charName (GROUP_CONCAT(DISTINCT ?typeLabel;separator=", ") AS ?types) (GROUP_CONCAT(DISTINCT ?universeLabel;separator=", ") AS ?universes)
         WHERE {
           ?char wdt:P1080 wd:Q931597;
                   wdt:P31 ?type ;
                   wdt:P1080 ?universe .
           SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en".
                                  ?char rdfs:label ?charName .
                                  ?universe rdfs:label ?universeLabel .
                                  ?type rdfs:label ?typeLabel .}
         } GROUP BY ?char ?charName
         LIMIT 500

                 
        """
query_results_Moovies = get_results(endpoint_url, query_results_Moovies)

for result in query_results_Moovies["results"]["bindings"]:
    print(result)
    
    print(result['label']["value"])
    print(result['date']["value"])

{'char': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q578279'}, 'charName': {'xml:lang': 'en', 'type': 'literal', 'value': 'Firestar'}, 'types': {'type': 'literal', 'value': 'mutant'}, 'universes': {'type': 'literal', 'value': 'Marvel Universe'}}


KeyError: 'label'

In [54]:

query_results_Moovies = """#train accident 
SELECT ?label ?coord ?date
WHERE
{
   ?subj wdt:P31 wd:Q744913  .
   ?subj wdt:P625 ?coord .
   ?subj rdfs:label ?label;
    wdt:P585  ?date. 
                 FILTER (lang(?label) = "en")
        }"""
query_results_Moovies = get_results(endpoint_url, query_results_Moovies)

for result in query_results_Moovies["results"]["bindings"]:
    #print(result)
    
    print(result[list(result.keys())[2]]["value"])
    print(result[list(result.keys())[0]]["value"])

    

China Airlines Flight 006
1985-02-19T00:00:00Z
China Airlines Flight 676
1998-02-16T00:00:00Z
South African Airways Flight 295
1987-11-28T00:00:00Z
EgyptAir Flight 11990
1999-10-31T00:00:00Z
2008 Los Roques archipelago Transaven Let L-410 crash
2008-01-04T00:00:00Z
1972 Andes flight disaster
1972-10-13T00:00:00Z
LAPA Flight 3142
1999-08-31T00:00:00Z
1977 Convair CV-300 crash
1977-10-20T00:00:00Z
Sol Líneas Aéreas Flight 5428
2011-05-18T00:00:00Z
Airlines PNG Flight 1600
2011-10-13T00:00:00Z
ČSA Flight 001
1976-07-28T00:00:00Z
Garuda Indonesia Flight 152
1997-09-26T00:00:00Z
2005 Royal Air Force Hercules shootdown
2005-01-30T00:00:00Z
Air Moorea Flight 1121
2007-08-09T00:00:00Z
Iran Aseman Airlines Flight 6895
2008-08-24T00:00:00Z
1996 Air Africa crash
1996-01-08T00:00:00Z
Iran Air Flight 655 shootdown
1988-07-03T00:00:00Z
American Airlines Flight 11
2001-09-11T00:00:00Z
United Airlines Flight 175
2001-09-11T00:00:00Z
Japan Airlines Flight 123
1985-08-12T00:00:00Z
Hindenburg disaster
19

Ozark Air Lines Flight 809
1973-07-23T00:00:00Z
PIA Flight 688
2006-07-10T00:00:00Z
PIA Flight 740
1979-11-26T00:00:00Z
Pacific Western Airlines Flight 314
1978-02-11T00:00:00Z
Pacific Western Airlines Flight 501
1984-03-22T00:00:00Z
Pacoima aircraft accident
1957-01-31T00:00:00Z
Widerøe Flight 933
1982-03-11T00:00:00Z
Widerøe Flight 710
1988-05-06T00:00:00Z
Linea Aeropostal Venezolana Flight 253 (June 1956)
1956-06-20T00:00:00Z
Linjeflyg Flight 618
1977-01-15T00:00:00Z
Los Angeles Airways Flight 417
1968-08-14T00:00:00Z
Lovettsville air disaster
1940-08-31T00:00:00Z
MacRobertson Miller Airlines Flight 1750
1968-12-31T00:00:00Z
Winter Hill air disaster
1958-02-27T00:00:00Z
1957 Aquila Airways Solent crash
1957-11-15T00:00:00Z
Northwest Airlines Flight 421
1948-08-29T00:00:00Z
1955 Altensteig mid-air collision
1955-08-11T00:00:00Z
Malaysian Airline System Flight 684
1983-12-18T00:00:00Z
Khabarovsk United Air Group Flight 3949
1995-12-06T00:00:00Z
2014 Algerian Air Force C-130 crash
2014

In [ ]:
class_dic['Movies']

# extra data

In [ ]:
df = pd.read_csv("./dataset_journal/flights_en.txt", sep="\t")
df.head(433)

In [ ]:
df['flight'][430:480]

In [ ]:
date=pd.to_datetime(df['date'])

In [ ]:
date[430:480]

# Event Series

In [21]:

query_event = """#Recent events
#title: Recent events
SELECT ?event ?eventLabel ?date
WITH {
  SELECT ?event ?date
  WHERE {
    # find events
    ?event wdt:P31/wdt:P279* wd:Q1190554.
    # with a point in time or start date
    OPTIONAL { ?event wdt:P585 ?date. }
    OPTIONAL { ?event wdt:P580 ?date. }
    # but at least one of those
    FILTER(BOUND(?date) && DATATYPE(?date) = xsd:dateTime).
    # not in the future, and not more than 3100cal days ago
    BIND(NOW() - ?date AS ?distance).
    FILTER(0 <= ?distance && ?distance < 13000).
  }
  LIMIT 20000
} AS %i
WHERE {
  INCLUDE %i
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en" . }
}"""

results = get_results(endpoint_url, query_event)



In [22]:
event_df = pd.DataFrame(columns=('eventLabel', 'date'))
i=0
for result in results["results"]["bindings"]:   
    
    event_df.loc[i] = [result['eventLabel']['value'],result['date']['value']]
    i=i+1

In [23]:
event_df

,eventLabel,date
0,Q19394126,2008-01-01T00:00:00Z
1,Q19394108,1999-01-01T00:00:00Z
2,Q19394107,1999-01-01T00:00:00Z
3,Q19388799,2003-01-01T00:00:00Z
4,Q19375867,1993-01-01T00:00:00Z
...,...,...
20001,Q55369826,2009-03-21T00:00:00Z
20002,Q55369829,2009-09-26T00:00:00Z
20003,Q55369831,2010-02-19T00:00:00Z
20004,Q55369832,2010-09-25T00:00:00Z


In [24]:
event_df_clean=event_df[~event_df.eventLabel.str.match('[Q](\d{4})')].reset_index(drop=True) #drop the Q/d/d/d....

In [25]:
event_df_clean['date'] = pd.to_datetime(event_df_clean['date'])

In [26]:
event_df_clean=event_df_clean.sort_values(by=['date'])

In [27]:
event_df_clean1=event_df_clean[~event_df_clean.duplicated()]

In [38]:
no_duplicates=event_df_clean1[~event_df_clean1.duplicated(['eventLabel'])]


In [39]:
no_duplicates

,eventLabel,date
4184,Operation Karbala-7,1987-03-08 00:00:00+00:00
3184,"The Mind's I, Part 2",1987-03-13 00:00:00+00:00
2498,Francis Hayman,1987-04-01 00:00:00+00:00
2319,Keiko Hasegawa,1987-04-15 00:00:00+00:00
6890,Banjarsari massacre,1987-04-15 00:00:00+00:00
...,...,...
3629,10-point plan to address the information crisis,2022-09-02 00:00:00+00:00
3426,Philosophy [in:of:for:and] Digital Knowledge I...,2022-09-08 00:00:00+00:00
3958,International Operational Innovation Conference,2022-09-12 00:00:00+00:00
3248,2022 European Sambo Championships,2022-09-14 00:00:00+00:00


In [40]:
no_duplicates[no_duplicates['eventLabel']=='death and funeral of Mikhail Gorbachev']

,eventLabel,date
3172,death and funeral of Mikhail Gorbachev,2022-08-30 00:00:00+00:00


In [41]:
event_df_clean1[event_df_clean1['eventLabel']=='death and funeral of Mikhail Gorbachev']

,eventLabel,date
3172,death and funeral of Mikhail Gorbachev,2022-08-30 00:00:00+00:00
3173,death and funeral of Mikhail Gorbachev,2022-09-03 00:00:00+00:00


In [42]:
no_duplicates.to_csv('event.csv',index=True) 

In [35]:
url = 'https://en.wikipedia.org/w/api.php'
params = {
                    'action': 'query',
                    'format': 'json',
                    'rvprop' : 'timestamp',
                    'titles': 'LAN Chile Flight 107',
                    'prop': 'revisions',
                    'rvdir' : 'older',
                    'rvlimit' : '10'   ,
                    'rvexcludeuser': True}

data = requests.get(url, params=params).json()
key=[i for i in data['query']['pages'].keys()]
data['query']['pages'][key[0]]['revisions'][0]['timestamp']

'2021-10-09T08:50:35Z'

In [36]:
data

{'batchcomplete': '',
 'query': {'pages': {'68938033': {'pageid': 68938033,
    'ns': 0,
    'title': 'LAN Chile Flight 107',
    'revisions': [{'timestamp': '2021-10-09T08:50:35Z'}]}}}}